In [ ]:
# In this notebook we explicitly calculate yearly event rates according to the run plan specified and cited in the paper.
# This can be used to produce norms.txt, which is required for ToysROOTtoNumpy.ipynb.
# Prerequisites:
# 1) xsecs.txt file containing scale factors from 58 different off-axis angles samples

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.random as rnd
from csv import reader
from scipy.interpolate import interp2d
import os
import uproot as uproot
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import scipy.stats
from sklearn import linear_model
from scipy import optimize
#import warnings; warnings.simplefilter('ignore')
from matplotlib.pyplot import figure

%jsroot on

In [ ]:
with uproot.open("Fluxes.ND.root") as fFluxes :
    oa_flux = fFluxes['LBNF_numu_flux'].values()
    energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
    angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians

oa_flux = oa_flux.transpose()
first_bin_flux = 8.9769702e-08
norm = (oa_flux[0].sum()/100)/first_bin_flux
oa_flux = oa_flux/norm
oa_flux = oa_flux.transpose()

In [ ]:
lar_dens = 1.3954 ##g/cm^3
lar_mass_g = lar_dens*(2*3*0.574)*1e6 ##g
lar_mass_kg = lar_mass_g*1e-3
nucleons = lar_mass_kg/1.66e-27
print(lar_mass_kg)
print(nucleons)

In [ ]:
xsecs_string = open("../XSec_ROOT/xsecs.txt", "r").read().split("\n")
cross_section_norm = 1
cross_section = cross_section_norm*1e-38*np.array([float(x) for x in xsecs_string])
#N_target = 1.435e30 #[nucleon]
N_target = nucleons
E = (12/201)*1.1e21 #[POT per year]
epsilon = 1
cross_section = np.tile(cross_section, (oa_flux.shape[0],1))
oa_events_original = oa_flux * cross_section * epsilon * E * N_target
events_vector = np.array([oa_events_original[:,k].sum()/100 for k in range(oa_events_original.shape[1])])
print(events_vector)
plt.plot(events_vector)

In [ ]:
def write_list_to_file(values, filename="output.txt"):
    """
    Writes the elements of a list to a .txt file, each on a new line.

    Args:
        values (list): The list of values to write.
        filename (str): The name of the output file (default: "output.txt").
    """
    try:
        with open(filename, "w") as file:
            for value in values:
                file.write(f"{value}\n")
        print(f"List successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

write_list_to_file(events_vector, "norms.txt")